# Giriş


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import pandas as pd

# Google Drive'daki klasörünüzün yolunu belirtin
drive_folder = '/content/drive/My Drive/BBCNewsSummary'

path_, filename_, category_, article_or_summary_ = [], [], [], []

for dirname, _, filenames in os.walk(drive_folder):
    for filename in filenames:
        path_.append(os.path.join(dirname, filename))
        filename_.append(filename)
        category_.append(dirname.split("/")[-1])
        article_or_summary_.append(dirname.split("/")[-2])

# Verileri bir DataFrame'e dönüştürüyoruz
df = pd.DataFrame({
    'Path': path_,
    'Filename': filename_,
    'Category': category_,
    'Article_or_Summary': article_or_summary_
})

# DataFrame'i görüntülüyoruz
print(df.head())


                                                Path Filename Category  \
0  /content/drive/My Drive/BBCNewsSummary/Summari...  236.txt     tech   
1  /content/drive/My Drive/BBCNewsSummary/Summari...  193.txt     tech   
2  /content/drive/My Drive/BBCNewsSummary/Summari...  080.txt     tech   
3  /content/drive/My Drive/BBCNewsSummary/Summari...  262.txt     tech   
4  /content/drive/My Drive/BBCNewsSummary/Summari...  324.txt     tech   

  Article_or_Summary  
0          Summaries  
1          Summaries  
2          Summaries  
3          Summaries  
4          Summaries  


In [3]:
df = pd.DataFrame({"path":path_, "filename":filename_, "category":category_, "article_or_summary":article_or_summary_}, columns=["path", "filename", "category", "article_or_summary"])
df

,path,filename,category,article_or_summary
0,/content/drive/My Drive/BBCNewsSummary/Summari...,236.txt,tech,Summaries
1,/content/drive/My Drive/BBCNewsSummary/Summari...,193.txt,tech,Summaries
2,/content/drive/My Drive/BBCNewsSummary/Summari...,080.txt,tech,Summaries
3,/content/drive/My Drive/BBCNewsSummary/Summari...,262.txt,tech,Summaries
4,/content/drive/My Drive/BBCNewsSummary/Summari...,324.txt,tech,Summaries
...,...,...,...,...
4445,/content/drive/My Drive/BBCNewsSummary/News Ar...,304.txt,business,News Articles
4446,/content/drive/My Drive/BBCNewsSummary/News Ar...,159.txt,business,News Articles
4447,/content/drive/My Drive/BBCNewsSummary/News Ar...,152.txt,business,News Articles
4448,/content/drive/My Drive/BBCNewsSummary/News Ar...,020.txt,business,News Articles


In [6]:
!pip install cufflinks plotly


# Keşifsel Veri Analizi


In [27]:
from collections import Counter
import pandas as pd

# "News Articles" kategorisine sahip verilerin sayılarını hesaplıyoruz
ct_news = Counter(df[df['article_or_summary'] == "News Articles"]["category"])
# DataFrame'i oluşturuyoruz
data_news = pd.DataFrame({"category": list(ct_news.keys()), "value": list(ct_news.values())})
print(data_news)
import plotly.express as px

# Bar grafiği oluşturma ve görüntüleme
fig_news = px.bar(data_news, x='category', y='value', title='Category Distribution in News Articles', labels={'category':'Category', 'value':'Value'})
fig_news.show(renderer="colab")


        category  value
0           tech    401
1          sport    511
2  entertainment    386
3       politics    417
4       business    510


In [30]:
from collections import Counter
import pandas as pd

# "Summaries" kategorisine sahip verilerin sayılarını hesaplama
ct_summaries = Counter(df[df['article_or_summary'] == "Summaries"]["category"])
# DataFrame'i oluşturma
data_summaries = pd.DataFrame({"category": list(ct_summaries.keys()), "value": list(ct_summaries.values())})
print(data_summaries)
import plotly.express as px

# Bar grafiği oluşturma ve görüntüleme
fig_summaries = px.bar(data_news, x='category', y='value', title='Category Distribution in Summaries', labels={'category':'Category', 'value':'Value'})
fig_summaries.show(renderer="colab")


        category  value
0           tech    401
1          sport    511
2       politics    417
3       business    510
4  entertainment    386


In [33]:
# "Summaries" ve "News Articles" kategorilerine sahip verilerin toplam sayılarını hesaplama
total_summaries = df[df['article_or_summary'] == "Summaries"].shape[0]
total_news = df[df['article_or_summary'] == "News Articles"].shape[0]

# Toplam verileri içeren bir veri çerçevesi oluşturma
data_totals = pd.DataFrame({
    "category": ["Summaries", "News Articles"],
    "count": [total_summaries, total_news]
})

print(data_totals)

fig_totals = px.pie(data_totals, names='category', values='count', title='Distribution of Summaries and News Articles')
fig_totals.show(renderer="colab")

        category  count
0      Summaries   2225
1  News Articles   2225


# Ön İşleme


***Cümle Tokenizasyonu***

In [35]:
import nltk # dil işleme için önemli bir kütüphanedir. Çeşitli özelliklere sahiptir. Natural Language Toolkit
from nltk.corpus import stopwords # stopwords, nltk kütüphanesinin bir parçasıdır ve genellikle anlam taşımayan, sık kullanılan kelimeleri içerir.
from nltk.tokenize import sent_tokenize # sent_tokenize, metni cümlelere bölmek (tokenize etmek) için kullanılan bir fonksiyondur.
import numpy as np
import networkx as nx
import re

In [37]:
# burada input olarak bir text alan ve bunu cümlelere bölen bir fonksiyon oluşturuyor.
def read_article(text):
    sentences =[]
    for sentence in sent_tokenize(text):
        sentence.replace("[^a-zA-Z0-9]"," ")
    return sent_tokenize(text)

In [38]:
# haber makalesi olanlardan ilkini alıyoruz.
file_path = df[df['article_or_summary']=='News Articles'].iloc[0]['path']
with open(file_path, "r") as f:
    article = f.read()

In [41]:

sent_tok = read_article(article)
sent_tok

["Peer-to-peer nets 'here to stay'\n\nPeer-to-peer (P2P) networks are here to stay, and are on the verge of being exploited by commercial media firms, says a panel of industry experts.",
 'Once several high-profile legal cases against file-sharers are resolved this year, firms will be very keen to try and make money from P2P technology.',
 'The expert panel probed the future of P2P at the Consumer Electronics Show in Las Vegas earlier in January.',
 'The first convictions for P2P piracy were handed out in the US in January.',
 'William Trowbridge and Michael Chicoine pleaded guilty to charges that they infringed copyright by illegally sharing music, movies and software.',
 'Since the first successful file-sharing network Napster was forced to close down, the entertainment industry has been nervous and critical of P2P technology, blaming it for falling sales and piracy.',
 'But that is going to change very soon, according to the panel.',
 'The music and film industries have started some

***Yazım Düzeltme***

In [43]:
#TextBlob kütüphanesini kullanarak cümleleri veya metin parçalarını dilbilgisel olarak düzeltmeye çalışıyoruz.
#TextBlob, metinleri analiz etmek ve işlemek için kullanılan bir Python kütüphanesidir. Metin düzeltme, dil analizi ve diğer işlemler için kullanılabilir.

from textblob import TextBlob
mod_sent = []
for tok in sent_tok:
    blob_obj = TextBlob(tok)
    correct_sent = str(blob_obj.correct())
    print(f"\033[94m Original Token : {tok} \033[0m")
    print(f"\033[92m Corrected Token: {correct_sent} \033[92m")
    mod_sent.append(correct_sent)

 Original Token : Peer-to-peer nets 'here to stay'

Peer-to-peer (P2P) networks are here to stay, and are on the verge of being exploited by commercial media firms, says a panel of industry experts. 
 Corrected Token: Beer-to-peer nets 'here to stay'

Beer-to-peer (P2P) network are here to stay, and are on the verge of being exploited by commercial media firms, says a panel of industry experts. 
 Original Token : Once several high-profile legal cases against file-sharers are resolved this year, firms will be very keen to try and make money from P2P technology. 
 Corrected Token: Once several high-profile legal cases against file-shares are resolved this year, firms will be very keen to try and make money from P2P technology. 
 Original Token : The expert panel probed the future of P2P at the Consumer Electronics Show in Las Vegas earlier in January. 
 Corrected Token: The expert panel probed the future of P2P at the Consumer Electronics How in Was Began earlier in January. 
 Original T

In [44]:
" ".join(mod_sent)

'Beer-to-peer nets \'here to stay\'\n\nBeer-to-peer (P2P) network are here to stay, and are on the verge of being exploited by commercial media firms, says a panel of industry experts. Once several high-profile legal cases against file-shares are resolved this year, firms will be very keen to try and make money from P2P technology. The expert panel probed the future of P2P at the Consumer Electronics How in Was Began earlier in January. The first convictions for P2P pray were handed out in the of in January. William Trowbridge and Michael Chicoine pleaded guilty to charges that they infringed copyright by illegally sharing music, moves and software. Since the first successful file-sharing network Master was forced to close down, the entertainment industry has been nervous and critical of P2P technology, blaming it for falling sales and pray. But that is going to change very soon, according to the panel. The music and film industries have started some big legal cases against owners of l

# Benzerlik Hesaplama

In [45]:
import tensorflow_hub as hub

#Burada cümleleri vektörlere çeviren Google Universal Encoder kullanıyoruz.
#GSE, cümleleri yüksek boyutlu vektörlere dönüştüren bir derin öğrenme modelidir ve benzerlik hesaplamalarında yaygın olarak kullanılır.

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

#Burada 2 cümle arasındaki benzerlik skorunu hesaplayan bir fonksiyon oluşturuyoruz.

def sentence_similarity(sent1,sent2,embed):
    A = embed([sent1])[0]
    B = embed([sent2])[0]
    return 1 - (np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B)))

In [46]:
print(f"\033[92m Sentence 1 : {mod_sent[0]}")
print(f"\033[92m Sentence 2 : {mod_sent[1]}")
print(f"\033[92m Similarity Score : {sentence_similarity(mod_sent[0], mod_sent[1], embed)}")

 Sentence 1 : Beer-to-peer nets 'here to stay'

Beer-to-peer (P2P) network are here to stay, and are on the verge of being exploited by commercial media firms, says a panel of industry experts.
 Sentence 2 : Once several high-profile legal cases against file-shares are resolved this year, firms will be very keen to try and make money from P2P technology.
 Similarity Score : 0.6436501741409302


In [47]:
#Özetleme işlemi sırasında, tüm cümle çiftleri arasındaki benzerlik skorlarını hesaplamak ve bunları mesafe matrisi olarak saklamak gereklidir.
#Bu matriste her bir hücre, iki cümle arasındaki benzerlik skorunu temsil eder.

def build_similarity_matrix(sentences,embeds):
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],embeds)
    return similarity_matrix

In [48]:
sim_mat = build_similarity_matrix(mod_sent, embed)

In [51]:
from bokeh.io import output_notebook, show
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
import networkx as nx

output_notebook()

# Ağ grafiği oluşturma
g = nx.Graph()

# Sim matrisini kullanarak kenarları ekliyoruz.
for i in range(sim_mat.shape[0]):
    for j in range(sim_mat.shape[1]):
        if sim_mat[i][j] >= 0.9:
            g.add_edge(i, j)

HOVER_TOOLTIPS = [("sent_tok", "@index")]
plot = figure(tooltips=HOVER_TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom', x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1))

# NetworkX'den Bokeh'e ağ grafiğini aktarma
network_graph = from_networkx(g, nx.spring_layout, scale=7, center=(0, 0))

# Düğüm ve kenar özelliklerini ayarlama
network_graph.node_renderer.glyph = Circle(radius=15, fill_color='green')  # 'size' yerine 'radius' kullanın
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

# Grafik üzerinde ağ grafiğini görüntüleme
plot.renderers.append(network_graph)
show(plot)


# Metin Özetleme

In [52]:

#Summaries kısmındaki ilk değeri alıyoruz.
file_path_summary = df[df['article_or_summary']=='Summaries'].iloc[0]['path']
with open(file_path_summary, "r") as f:
    actual_summary = f.read()

In [53]:
#burada özet oluşturacak metodumuzu oluşturuyoruz.
#Girdi olarak asıl yazıyı, kaç cümle olacağını, ve vektörü alıyor.
#read article metodu ile tokenize ediyoruz. Daha sonrasında matrisini oluşturuyoruz.
#skorları grafta yansıtıyoruz ve en yüksek skor alanlarla özetimizi oluşturuyoruz.

def generate_summary(text,top_n,embeds):
    summarize_text = []
    sentences = read_article(text)
    sentence_similarity_matrix = build_similarity_matrix(sentences,embeds)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    return " ".join(summarize_text)

In [54]:
#Bir örneği burada gösterelim.
Original_Text = " ".join(mod_sent)
Summarized_Text = generate_summary(Original_Text, top_n=5, embeds=embed)

In [55]:
Original_Text

'Beer-to-peer nets \'here to stay\'\n\nBeer-to-peer (P2P) network are here to stay, and are on the verge of being exploited by commercial media firms, says a panel of industry experts. Once several high-profile legal cases against file-shares are resolved this year, firms will be very keen to try and make money from P2P technology. The expert panel probed the future of P2P at the Consumer Electronics How in Was Began earlier in January. The first convictions for P2P pray were handed out in the of in January. William Trowbridge and Michael Chicoine pleaded guilty to charges that they infringed copyright by illegally sharing music, moves and software. Since the first successful file-sharing network Master was forced to close down, the entertainment industry has been nervous and critical of P2P technology, blaming it for falling sales and pray. But that is going to change very soon, according to the panel. The music and film industries have started some big legal cases against owners of l

In [56]:
Summarized_Text

'"Mile-sharing is the tip of the iceberg." But that is going to change very soon, according to the panel. "When they can\'t stop it, they try to control it. But the technology is still demolished and misunderstood by many. When they figure out how to make money and they always make a lot of money."'

In [57]:
actual_summary

'Online pages from scanned books will not have adverts but will have links to online store Amazon, Google said.The New York library is allowing Google to include a small portion of books no longer covered by copyright.The full libraries of Michigan and Stanford universities, as well as archives at Harvard, Oxford and the New York Public Library are included."We hope that Oxford\'s contribution to this project will be of scholarly use, as well as general interest, to people around the world," said Reg Carr, director of Oxford University Library Services.Harvard is limiting its participation to 40,000 books, while Oxford wants Google to scan books originally published in the 19th Century and held in the Bodleian Library.There will also be links to public libraries so that the books can be borrowed."It\'s a significant opportunity to bring our material to the rest of the world," said Paul LeClerc, president of the New York Public Library.About one million books will be scanned by Google, 

In [58]:
import nltk
# Burada doğrulama için BLEU skoru kullanıyoruz
# Bu metrik, model tarafından üretilen metinlerin referans metinlere ne kadar benzediğini ölçer.
# N-Gram karşılaştırılmaları yaparak, metinlerin benzerlik oranını belirler.
hypothesis = Summarized_Text
reference = actual_summary
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(f"BLEUscore : {BLEUscore}")

BLEUscore : 0.04110429632608801


In [59]:
print(f"Senetence Similarity Score : {sentence_similarity(Summarized_Text, actual_summary, embed)}")

Senetence Similarity Score : 0.9613198526203632


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.9 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=111a0d71d962f20a6c26da5e2719a926a27424b78f0341bbeef3a1126fb77df6
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=c6bde5fdf0365dbb6cf9c0e98cb82cb4bec89c4c85fc96d15cda4bf2d21cab66
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt
